In [1]:
# import libs 
import tensorflow as tf # for deep learning 

# helper libs 
import numpy as np # for matrix multiplication 

In [2]:
# import datset 
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('./data/MNIST/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# helper funtion for variables 
def weights_variable(shape):
    inital = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(inital, name="W")

def biases_variable(shape):
    inital = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(inital, name="B")

def FC_layer(X, W, B):
    return tf.matmul(X,W) + B

In [4]:
# define placeholder for input as x 
pixels_dim = 28 * 28

# input placeholder for image
X = tf.placeholder(tf.float32, shape=([None, pixels_dim]))

# a summary for the x_image 
tf.summary.image("Input", tf.reshape(X,[-1, 28,28, 1]), 3)

<tf.Tensor 'Input:0' shape=() dtype=string>

In [5]:
# encoder part 
latent_dim = 20
h_dim = 500

# layer 1
with tf.name_scope("encoder"):
    w_enc = weights_variable([pixels_dim, h_dim])
    b_enc = biases_variable([h_dim])
    
    h_enc = tf.nn.tanh(FC_layer(X, w_enc, b_enc))
    
    # write summary for the encoder 
    tf.summary.histogram("weights", w_enc)
    tf.summary.histogram("biases", b_enc)
    tf.summary.histogram("activation", h_enc)
    
# layer 2 
with tf.name_scope("mean"):
    w_mu = weights_variable([h_dim, latent_dim])
    b_mu = biases_variable([latent_dim])
    
    mu = FC_layer(h_enc, w_mu, b_mu) # mean
    
    # write summary for the mean values 
    tf.summary.histogram("weights", w_mu)
    tf.summary.histogram("biases",b_mu)
    
# layer 3 for std 
with tf.name_scope("std"):
    w_std = weights_variable([h_dim, latent_dim])
    b_std = biases_variable([latent_dim])
    
    logstd = FC_layer(h_enc, w_std, b_std)
    
    # write summary for the model 
    tf.summary.histogram("weights", w_std)
    tf.summary.histogram("biases", b_std)
    
# final output layer 
with tf.name_scope("enc_noise"):
    noise = tf.random_normal(shape=[1, latent_dim])
    
    z = mu + tf.multiply(noise, tf.exp(-5 * logstd))
    
    tf.summary.histogram("output", z)

In [6]:
# decoder part 

with tf.name_scope("decoder"):
    w_dec = weights_variable([latent_dim,h_dim])
    b_dec = biases_variable([h_dim])
    
    h_dec = tf.nn.tanh(FC_layer(z, w_dec, b_dec))
    
    # writing summary 
    tf.summary.histogram("weights", w_dec)
    tf.summary.histogram("biases", b_dec)
    tf.summary.histogram("activation", h_dec)

with tf.name_scope("reconstruct"):
    w_rec = weights_variable([h_dim, pixels_dim])
    b_rec = biases_variable([pixels_dim])
    
    # activation using sigmoid 
    reconstruction = tf.nn.sigmoid(FC_layer(h_dec, w_rec, b_rec))
    
    # summary for reconstruct
    tf.summary.histogram("weights", w_rec)
    tf.summary.histogram("biases", b_rec)
    tf.summary.histogram("activation", reconstruction)
    tf.summary.image("output", tf.reshape(reconstruction,[-1, 28,28, 1]))

In [7]:
# handelling loss functions 
with tf.name_scope("loss"):
    log_likelihood = tf.reduce_sum(X*tf.log(reconstruction + 1e-9)+(1 - X)*tf.log(1 - reconstruction + 1e-9), 
                                   reduction_indices=1)
    
    kl_term = -.5*tf.reduce_sum(1 + 2*logstd - tf.pow(mu,2) - tf.exp(2*logstd), reduction_indices=1)
    
    variation_lower_bound = tf.reduce_mean(log_likelihood - kl_term)
    
    # summary for these log terms 
    tf.summary.scalar("variational_lower_bound", variation_lower_bound)

with tf.name_scope("train"):
    optimizer = tf.train.AdadeltaOptimizer(learning_rate=1e-4).minimize(-variation_lower_bound)

In [8]:
# training step 
# init all the variables and sessions 
sess = tf.Session()

# init all the variables 
sess.run(tf.global_variables_initializer())

# init a saver 
saver = tf.train.Saver()

In [9]:
# write summaries 
summ = tf.summary.merge_all()

# make a summary writer 
writer = tf.summary.FileWriter('./logs')
# writing gaph for the session
writer.add_graph(sess.graph)

In [13]:
num_iterations = 1000000
recording_interval = 1000
#store value for these 3 terms so we can plot them later
# variational_lower_bound_array = []
# log_likelihood_array = []
# KL_term_array = []
# iteration_array = [i*recording_interval for i in range(num_iterations/recording_interval)]
for i in range(num_iterations):
    # np.round to make MNIST binary
    #get first batch (200 digits)
    x_batch = mnist.train.next_batch(200)[0]
    #run our optimizer on our data
    _, s =sess.run([optimizer, summ], feed_dict={X: x_batch})
    writer.add_summary(s, i)
    if (i%recording_interval == 0):
        #every 1K iterations record these values
        vlb_eval = sess.run(variation_lower_bound, feed_dict={X:x_batch})
        print("Iteration: {}, Loss: {}".format(i, vlb_eval))
#         variational_lower_bound_array.append(vlb_eval)
#         log_likelihood_array.append(np.mean(log_likelihood.eval(feed_dict={X: x_batch})))
#         KL_term_array.append(np.mean(KL_term.eval(feed_dict={X: x_batch})))

Iteration: 0, Loss: -1097.64453125


KeyboardInterrupt: 